In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from matplotlib import pyplot as plt
import keract
from tqdm import tqdm
import pandas as pd

import json

import tensorflow_model_optimization as tfmot


with_gpu = True
if not with_gpu:
    tf.config.experimental.set_visible_devices([], "GPU")
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("GPU is Enabled")
else:
    print("GPU is not Enabled")
    

GPU is not Enabled


In [5]:
# Step 1: Load and preprocess ImageNet dataset
def preprocess(img, label):
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    return img, label

ds_train = tfds.load(
    'imagenet2012',
    split='train',
    as_supervised=True,
    data_dir='/home/insane/U/NN Activation/imagenet'
)

ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.batch(32).prefetch(tf.data.AUTOTUNE)

ds_test = tfds.load(
    'imagenet2012',
    split='validation',
    as_supervised=True,
    data_dir='/home/insane/U/NN Activation/imagenet'
)

ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(32).prefetch(tf.data.AUTOTUNE)


In [7]:
# Step 2: Define the model BEFORE applying pruning
base_model = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



# Magnitude-based Weight Pruning

In [4]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(1000, activation='softmax')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

# Step 3: Apply pruning wrapper BEFORE compiling or calling the model
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=10000
    )
}
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# Step 4: Compile the model
model_for_pruning.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Step 5: Train with pruning callback
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
model_for_pruning.fit(
    ds_train,
    epochs=2,
    steps_per_epoch=100,
    callbacks=callbacks
)


# Save the pruned model
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# Or save in TensorFlow SavedModel format
model_for_export.save('pruned_vgg16_savedmodel')


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Epoch 1/2
100/100 [==============================] - 650s 6s/step - loss: 7.2837 - accuracy: 9.3750e-04
Epoch 2/2
100/100 [==============================] - 648s 6s/step - loss: 6.9076 - accuracy: 0.0016


INFO:tensorflow:Assets written to: pruned_vgg16_savedmodel/assets


INFO:tensorflow:Assets written to: pruned_vgg16_savedmodel/assets


In [9]:
conv_layers = [layer for layer in base_model.layers if isinstance(layer, tf.keras.layers.Conv2D)]

def l1_norm_filter_selection(weights, percent=0.3):
    """
    Select filter indices to keep based on L1-norm pruning.
    """
    filter_weights = weights[0]  # shape: (H, W, in_channels, out_channels)
    l1_norms = np.sum(np.abs(filter_weights), axis=(0, 1, 2))  # L1-norm per filter
    num_filters = filter_weights.shape[-1]
    num_prune = int(percent * num_filters)
    keep_indices = np.argsort(l1_norms)[num_prune:]  # keep the highest ones
    return keep_indices

def prune_conv_weights(weights, input_keep_indices, output_keep_indices):
    """
    Prune weights based on selected input and output channel indices.
    """
    W, b = weights
    # Prune input channels (axis=2) and output filters (axis=3)
    W = W[:, :, input_keep_indices, :]
    W = W[:, :, :, output_keep_indices]
    b = b[output_keep_indices]
    return [W, b]

def build_pruned_model(percent=0.3):
    input_tensor = tf.keras.layers.Input(shape=(224, 224, 3))
    x = input_tensor

    previous_keep_indices = np.arange(3)  # RGB input

    pruned_weights = []
    keep_indices_list = []

    # Analyze pruning first
    for layer in conv_layers:
        orig_weights = layer.get_weights()
        if not orig_weights:
            continue
        output_keep_indices = l1_norm_filter_selection(orig_weights, percent)
        keep_indices_list.append((previous_keep_indices, output_keep_indices))
        pruned_w = prune_conv_weights(orig_weights, previous_keep_indices, output_keep_indices)
        pruned_weights.append(pruned_w)
        previous_keep_indices = output_keep_indices

    # Build new model
    layer_idx = 0
    previous_filters = 3
    previous_keep_indices = np.arange(3)
    for layer in base_model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            in_idx, out_idx = keep_indices_list[layer_idx]
            filters = len(out_idx)
            x = tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=layer.activation,
                name=f"pruned_conv_{layer_idx}"
            )(x)
            layer_idx += 1
        elif isinstance(layer, tf.keras.layers.MaxPooling2D):
            x = tf.keras.layers.MaxPooling2D(pool_size=layer.pool_size, strides=layer.strides, padding=layer.padding)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1000, activation='softmax')(x)
    pruned_model = tf.keras.Model(inputs=input_tensor, outputs=x)

    # Set pruned weights
    conv_idx = 0
    for layer in pruned_model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            layer.set_weights(pruned_weights[conv_idx])
            conv_idx += 1

    return pruned_model

# Build and compile the pruned model
pruned_model = build_pruned_model(percent=0.3)
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

pruned_model.fit(ds_train, epochs=5, steps_per_epoch=100)


# Save the pruned model
pruned_model.save("vgg16_filter_pruned.h5")


Epoch 1/5
100/100 [==============================] - 407s 4s/step - loss: 6.9168 - accuracy: 6.2500e-04
Epoch 2/5
100/100 [==============================] - 390s 4s/step - loss: 6.9080 - accuracy: 3.1250e-04
Epoch 3/5
100/100 [==============================] - 376s 4s/step - loss: 6.9079 - accuracy: 9.3750e-04
Epoch 4/5
100/100 [==============================] - 380s 4s/step - loss: 6.9078 - accuracy: 0.0012
Epoch 5/5
100/100 [==============================] - 387s 4s/step - loss: 6.9087 - accuracy: 0.0012


In [11]:
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet')
base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load the original and pruned models for comparison
weight_pruned_model = tf.keras.models.load_model("../../saved models/vgg16_filter_pruned.h5")
weight_pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

filter_pruned_model = tf.keras.models.load_model("../../saved models/pruned_vgg16_savedmodel")
filter_pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Compare number of parameters
print("Number of parameters:")
print(f"Base model: {base_model.count_params():,}")
print(f"Weight pruned model: {weight_pruned_model.count_params():,}")
print(f"Filter pruned model: {filter_pruned_model.count_params():,}")

print("\nEvaluating models on test data...")

# Evaluate models on test data
base_results = base_model.evaluate(ds_test, steps=50, verbose=0)
weight_pruned_results = weight_pruned_model.evaluate(ds_test, steps=50, verbose=0)
filter_pruned_results = filter_pruned_model.evaluate(ds_test, steps=50, verbose=0)

print("\nTest Performance:")
print(f"Base model - Loss: {base_results[0]:.4f}, Accuracy: {base_results[1]:.4f}")
print(f"Weight pruned model - Loss: {weight_pruned_results[0]:.4f}, Accuracy: {weight_pruned_results[1]:.4f}")
print(f"Filter pruned model - Loss: {filter_pruned_results[0]:.4f}, Accuracy: {filter_pruned_results[1]:.4f}")

Number of parameters:
Base model: 138,357,544
Weight pruned model: 7,601,979
Filter pruned model: 15,227,688

Evaluating models on test data...

Test Performance:
Base model - Loss: 1.4519, Accuracy: 0.6600
Weight pruned model - Loss: 6.9082, Accuracy: 0.0044
Filter pruned model - Loss: 6.9087, Accuracy: 0.0012
